# Softmax 回归从零开始实现

In [ ]:
%matplotlib inline

In [ ]:
from visualize import use_svg_display

use_svg_display()

In [ ]:
BATCH_SIZE = 256
NUM_INPUTS = 784
NUM_OUTPUTS = 10
LR = 0.1
NUM_EPOCHS = 10

## Step1 数据集

In [ ]:
from torchvision.transforms import ToTensor
from torchvision.datasets import FashionMNIST

trans = ToTensor()

mnist_train = FashionMNIST(root="../data", train=True, transform=trans, download=True)
mnist_test = FashionMNIST(root="../data", train=False, transform=trans, download=True)

In [ ]:
len(mnist_train), len(mnist_test)

In [ ]:
mnist_train[0][0].shape

In [ ]:
def get_fashion_mnist_labels(labels):
    """
    返回 Fashion-MNIST 数据集的文本标签
    """
    text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
    
    return [text_labels[int(i)] for i in labels]

In [ ]:
from torch.utils.data import DataLoader
from visualize import show_images

X, y = next(iter(DataLoader(mnist_train, batch_size=18)))

show_images(X.reshape(18, 28, 28), 2, 9, titles=get_fashion_mnist_labels(y))

In [ ]:
train_iter = DataLoader(mnist_train, BATCH_SIZE, shuffle=True)

In [ ]:
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
from torchvision.transforms import Compose, Resize, ToTensor

def load_data_fashion_mnist(batch_size, resize=None):
    """
    下载 Fashion-MNIST 数据集，然后将其加载到内存中
    """
    trans = [ToTensor()]

    if resize:
        trans.insert(0, Resize(resize))
    
    trans = Compose(trans)

    mnist_train = FashionMNIST(root="../data", train=True, transform=trans, download=True)
    mnist_test = FashionMNIST(root="../data", train=False, transform=trans, download=True)
    
    return (
        DataLoader(mnist_train, batch_size, shuffle=True),
        DataLoader(mnist_test, batch_size, shuffle=False)
    )

In [ ]:
train_iter, test_iter = load_data_fashion_mnist(32, resize=64)

for X, y in train_iter:
    print(X.shape, X.dtype, y.shape, y.dtype)
    break

In [ ]:
train_iter, test_iter = load_data_fashion_mnist(BATCH_SIZE)

## Step2 定义模型

In [ ]:
import torch

W = torch.normal(0, 0.01, size=(NUM_INPUTS, NUM_OUTPUTS), requires_grad=True)
b = torch.zeros(NUM_OUTPUTS, requires_grad=True)

In [ ]:
import torch

def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1, keepdim=True)
    return X_exp / partition

In [ ]:
def net(X):
    return softmax(X.reshape((-1, W.shape[0])) @ W + b)

## Step3 定义损失函数

In [ ]:
def cross_entropy(y_hat, y):
    return -torch.log(y_hat[range(len(y_hat)), y])

In [ ]:
def accuracy(y_hat, y):
    """
    计算预测正确的数量
    """
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    
    cmp = y_hat.type(y.dtype) == y
    
    return float(cmp.type(y.dtype).sum())

In [ ]:
from utils import Accumulator

def evaluate_accuracy(net, data_iter):
    """
    计算在指定数据集上模型的精度
    """
    if isinstance(net, torch.nn.Module):
        net.eval()
    
    metric = Accumulator(2)  # 正确预测数、预测总数
    
    with torch.no_grad():
        for X, y in data_iter:
            metric.add(accuracy(net(X), y), y.numel())
    
    return metric[0] / metric[1]

## Step4 训练

In [ ]:
def train_epoch(net, train_iter, loss, updater):
    """
    训练模型一个迭代周期
    """
    if isinstance(net, torch.nn.Module):
        net.train()
    
    metric = Accumulator(3)  # 训练损失总和、训练准确度总和、样本数
    
    for X, y in train_iter:
        y_hat = net(X)
        l = loss(y_hat, y)
        
        if isinstance(updater, torch.optim.Optimizer):
            # 使用 PyTorch 内置的优化器和损失函数
            updater.zero_grad()
            l.mean().backward()
            updater.step()
        else:
            # 使用自定义的优化器和损失函数
            l.sum().backward()
            updater(X.shape[0])
        
        metric.add(float(l.sum()), accuracy(y_hat, y), y.numel())
    
    # 返回训练损失和训练精度
    return metric[0] / metric[2], metric[1] / metric[2]

In [ ]:
from visualize import Animator

def train(net, train_iter, test_iter, loss, num_epochs, updater):
    """
    训练模型
    """
    animator = Animator(
        xlabel='epoch', 
        xlim=[1, num_epochs], 
        ylim=[0.3, 0.9],
        legend=['train loss', 'train acc', 'test acc']
    )
    
    for epoch in range(num_epochs):
        train_metrics = train_epoch(net, train_iter, loss, updater)
        test_acc = evaluate_accuracy(net, test_iter)
        animator.add(epoch + 1, train_metrics + (test_acc,))
    
    train_loss, train_acc = train_metrics
    
    assert train_loss < 0.5, train_loss
    assert train_acc <= 1 and train_acc > 0.7, train_acc
    assert test_acc <= 1 and test_acc > 0.7, test_acc

In [ ]:
from trainer import sgd

def updater(batch_size):
    return sgd([W, b], LR, batch_size)

In [ ]:
train(net, train_iter, test_iter, cross_entropy, NUM_EPOCHS, updater)

## Step5 预测

In [ ]:
def predict(net, test_iter, n=6):
    """
    预测标签
    """
    for X, y in test_iter:
        break
    
    labels = get_fashion_mnist_labels(y)
    preds = get_fashion_mnist_labels(net(X).argmax(axis=1))
    titles = [true +'\n' + pred for true, pred in zip(labels, preds)]
    
    show_images(X[0:n].reshape((n, 28, 28)), 1, n, titles=titles[0:n])

predict(net, test_iter)

# Softmax 回归 PyTorch 实现

## Step1 数据集

In [ ]:
train_iter, test_iter = load_data_fashion_mnist(BATCH_SIZE)

## Step2 定义模型

In [ ]:
from torch import nn

net = nn.Sequential(
    nn.Flatten(), 
    nn.Linear(784, 10)
)

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights)

## Step3 定义损失函数

In [ ]:
loss = nn.CrossEntropyLoss(reduction='none')

## Step4 定义优化算法

In [ ]:
trainer = torch.optim.SGD(net.parameters(), lr=0.1)

In [ ]:
train(net, train_iter, test_iter, loss, NUM_EPOCHS, trainer)